In [73]:
#Imports

from scipy.optimize import minimize

In [74]:
Cinlet = 20 #mole/L
Vtot = 20 #m3
q = 0.03 #m3/s
n = 2.4
k = 0.0062 #(mole/l)**-n/s
V2 = 10
V1 = Vtot - V2

In [75]:
def obj_func(Cin, Coutlet):
    return -Coutlet/Cin
"""
descision_variables = [V1, V2]

constraints = ({'type': 'eq', 'fun': lambda x: x[0] + x[1] - Vtot})

sol = minimize(obj_func, descision_variables, constraints=constraints)
"""

"\ndescision_variables = [V1, V2]\n\nconstraints = ({'type': 'eq', 'fun': lambda x: x[0] + x[1] - Vtot})\n\nsol = minimize(obj_func, descision_variables, constraints=constraints)\n"

In [76]:

import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import minimize, fsolve
from scipy.integrate import solve_ivp
def cstrobjective(V,params):
    # decision varaiables, V(1:N-1)
    params['V'] = np.hstack(( V, params['Vtot']-np.sum(V) ))
    cout = cstrsolve(params)
    # maximze conversion
    conv = 1-cout/params['c0']
    # objective: minimize 1-conversion
    obj = 1-conv
    return obj
def cstrsolve(params):
    # solve for steady-state
    cinit = 0.5*params['c0']*np.ones(params['V'].shape)
    #print(f"cinit: {cinit}")
    cstat = fsolve(lambda c: cstrmodel(0,c,params), cinit)
    print(f"cstat: {cstat}")
    # outlet conc.
    cout = cstat[-1]
    return cout
def cstrmodel(t,c,params):
    # reaction rate
    r = params['k']*c**params['n']
    #print(f"r: {r}")
    # material balances
    cm1 = np.hstack(( params['c0'],c[:-1] ))
    #print(f"cm1: {cm1}")
    dcdt = params['q']/params['V']*(cm1 - c) - r    
    #print(f"dcdt: {dcdt}")
    return dcdt
params = {}
#number of cstr
params['N'] = 2
# total cstr series volume
params['Vtot']=20
# model paramsameters
params['c0'] = 20
params['n'] = 2.4
params['k'] = 0.0062
params['q'] = 0.03
Vguess = 10

In [77]:
#Unconstraint optimizaiton
# optimization using Nelder-Mead
print('==== Nelder-Mead ====')
optim1 = {'disp':True}
res1 = minimize(lambda V: cstrobjective(V,params), Vguess, method='Nelder-Mead', options=optim1)
Vf1 = res1['x']
print(f'Nelder-Mead - x: {res1["x"]}')

==== Nelder-Mead ====
cstat: [2.43893472 0.88740306]
cstat: [2.39248543 0.89362116]
cstat: [2.48867798 0.88223728]
cstat: [2.5421385  0.87809634]
cstat: [2.66233428 0.87286471]
cstat: [2.80498697 0.87184196]
cstat: [3.19632954 0.88488016]
cstat: [2.66233428 0.87286471]
cstat: [2.97845177 0.87551151]
cstat: [2.73041848 0.87180653]
cstat: [2.66233428 0.87286471]
cstat: [2.76682699 0.87168371]
cstat: [2.80498697 0.87184196]
cstat: [2.74841252 0.87171052]
cstat: [2.78567906 0.87172713]
cstat: [2.75756615 0.8716884 ]
cstat: [2.77619721 0.87169657]
cstat: [2.76218303 0.87168387]
cstat: [2.77149829 0.87168794]
cstat: [2.76450161 0.87168324]
cstat: [2.76218303 0.87168387]
cstat: [2.76566344 0.87168334]
cstat: [2.76334147 0.87168342]
cstat: [2.76508231 0.87168326]
cstat: [2.76392133 0.8716833 ]
cstat: [2.76479191 0.87168324]
cstat: [2.76508231 0.87168326]
cstat: [2.76464674 0.87168324]
cstat: [2.76450161 0.87168324]
cstat: [2.76471932 0.87168324]
cstat: [2.76457417 0.87168324]
cstat: [2.7646830

C:\Users\olofs\AppData\Local\Temp\ipykernel_20184\1318303029.py:25: RuntimeWarning: invalid value encountered in power
  r = params['k']*c**params['n']


In [78]:
# optimization using BFGS
print('==== BFGS ====')
optim2 = {'disp':True}
res2 = minimize(lambda V: cstrobjective(V,params), Vguess, method='BFGS', options=optim2)
Vf2 = res2['x']
print(f'BFGS - x: {res2["x"]}')

==== BFGS ====
cstat: [2.43893472 0.88740306]
cstat: [2.43893472 0.88740306]
cstat: [2.43898934 0.88739659]
cstat: [2.43898934 0.88739659]
cstat: [2.43920788 0.88737074]
cstat: [2.43920787 0.88737074]
cstat: [2.44008269 0.88726757]
cstat: [2.44008269 0.88726757]
cstat: [2.4435929  0.88685836]
cstat: [2.4435929  0.88685836]
cstat: [2.45781147 0.88527685]
cstat: [2.45781147 0.88527685]
cstat: [2.51771198 0.87982478]
cstat: [2.51771198 0.87982478]
cstat: [2.7518704 0.8717001]
cstat: [2.7518704 0.8717001]
cstat: [2.76561972 0.87168333]
cstat: [2.76561971 0.87168333]
Optimization terminated successfully.
         Current function value: 0.043584
         Iterations: 3
         Function evaluations: 18
         Gradient evaluations: 9
BFGS - x: [7.25810633]


C:\Users\olofs\AppData\Local\Temp\ipykernel_20184\1318303029.py:25: RuntimeWarning: invalid value encountered in power
  r = params['k']*c**params['n']
